In [1]:
import json
import sqlite3
import pandas as pd

### Преобразование данных

In [2]:
with open('./quotes.json', 'r') as f:
    data = json.load(f)

In [3]:
df_data = pd.DataFrame(data)
df_data.head(5)

,author,text,tags
0,Albert Einstein,“The world as we have created it is a process ...,"[change, deep-thoughts, thinking, world]"
1,J.K. Rowling,"“It is our choices, Harry, that show what we t...","[abilities, choices]"
2,Albert Einstein,“There are only two ways to live your life. On...,"[inspirational, life, live, miracle, miracles]"
3,Jane Austen,"“The person, be it gentleman or lady, who has ...","[aliteracy, books, classic, humor]"
4,Marilyn Monroe,"“Imperfection is beauty, madness is genius and...","[be-yourself, inspirational]"


In [4]:
# авторы, которые встречаются более 1 раза
set(df_data.loc[df_data['author'].duplicated(keep=False), 'author'])

{'Albert Einstein',
 'Bob Marley',
 'C.S. Lewis',
 'Charles Bukowski',
 'Dr. Seuss',
 'Eleanor Roosevelt',
 'Ernest Hemingway',
 'George R.R. Martin',
 'J.K. Rowling',
 'Jane Austen',
 'Marilyn Monroe',
 'Mark Twain',
 'Mother Teresa',
 'Ralph Waldo Emerson',
 'Suzanne Collins'}

> `Цитаты`

In [5]:
quotes_lst = [q['text'] for q in data]
len(quotes_lst)

100

In [6]:
tpl_quotes = []
for i, q in enumerate(quotes_lst):
    tpl_quotes.append((i, q))

> `Авторы`

In [7]:
authors_set = set([q['author'] for q in data])
len(authors_set)

50

In [8]:
tpl_authors = []
for i, q in enumerate(authors_set):
    tpl_authors.append((i, q))

> `Тэги`

In [9]:
# тэги
tags_set = set([t_lst for sublist in [q['tags'] for q in data] for t_lst in sublist])
len(tags_set)

137

In [10]:
tpl_tags = []
for i, q in enumerate(tags_set):
    tpl_tags.append((i, q))

> объединение `quotes и authors` по id

In [11]:
tpl_quotes_authors = []
for i, author in enumerate(authors_set):
    for j in df_data.loc[df_data['author'] == author, 'text'].index:
        tpl_quotes_authors.append((j, i))

> объединение `quotes и tags` по id

In [12]:
tpl_quotes_tags = []
tags_series = df_data['tags']

for i, t in enumerate(tags_set):
    mask = tags_series.str.contains(t, regex=False)
    
    for j in df_data.loc[mask, ['text', 'tags']].index:
        tpl_quotes_tags.append((j, i))
        
print(len(tpl_quotes_tags))

232


### Создание базы данных

In [13]:
def execute_query(q, crs, conn, type='simple query', tpl=None):
    if type == 'insert':
        crs.execute(q, tpl)  
    elif type == 'insert many':
        crs.executemany(q, tpl)
    elif type == 'script':
        crs.executescript(q)  
    else:
        crs.execute(q)
    conn.commit()

#### Структура базы

In [14]:
conn = sqlite3.connect('quotes.db')
cursor = conn.cursor()

> создание таблиц

In [15]:
create_tables_query = '''
    CREATE TABLE IF NOT EXISTS quotes (
        id SERIAL PRIMARY KEY,
        text TEXT(255) NOT NULL UNIQUE
    );
    
    CREATE TABLE IF NOT EXISTS authors (
        id SERIAL PRIMARY KEY,
        name TEXT(100) NOT NULL UNIQUE
    );
    
    CREATE TABLE IF NOT EXISTS tags (
        id SERIAL PRIMARY KEY,
        tag TEXT(50) NOT NULL UNIQUE
    );
    
    -- Таблицы связи
    CREATE TABLE IF NOT EXISTS quotes_authors (
        quote_id INT UNSIGNED NOT NULL,
        author_id INT UNSIGNED NOT NULL,
        FOREIGN KEY (quote_id) REFERENCES quotes (id),
        FOREIGN KEY (author_id) REFERENCES authors (id)
    );
    
    CREATE TABLE IF NOT EXISTS quotes_tags (
        quote_id INT UNSIGNED NOT NULL,
        tag_id INT UNSIGNED NOT NULL,
        FOREIGN KEY (quote_id) REFERENCES quotes (id),
        FOREIGN KEY (tag_id) REFERENCES tags (id)
    );    
'''

In [16]:
execute_query(create_tables_query, cursor, conn, 'script')

#### Загрузка данных в базу

In [19]:
insert_quotes = '''
    INSERT INTO quotes(id, text)
    VALUES(?, ?)
'''

In [20]:
execute_query(insert_quotes, cursor, conn, 'insert many', tpl_quotes)

In [21]:
insert_authors = '''
    INSERT INTO authors(id, name)
    VALUES(?, ?)
'''

In [22]:
execute_query(insert_authors, cursor, conn, 'insert many', tpl_authors)

In [23]:
insert_tags = '''
    INSERT INTO tags(id, tag)
    VALUES(?, ?)
'''

In [24]:
execute_query(insert_tags, cursor, conn, 'insert many', tpl_tags)

In [25]:
insert_quotes_authors = '''
    INSERT INTO quotes_authors(quote_id, author_id)
    VALUES(?, ?)
'''

In [26]:
execute_query(insert_quotes_authors, cursor, conn, 'insert many', tpl_quotes_authors)

In [27]:
insert_quotes_tags = '''
    INSERT INTO quotes_tags(quote_id, tag_id)
    VALUES(?, ?)
'''

In [28]:
execute_query(insert_quotes_tags, cursor, conn, 'insert many', tpl_quotes_tags)

### Возможные запросы к базе

In [29]:
aa_quotes = '''
    SELECT q.text, a.name 
    FROM quotes q
        LEFT JOIN quotes_authors qa
            ON q.id = qa.quote_id
        LEFT JOIN authors a
            ON a.id = qa.author_id
        WHERE a.name = 'Albert Einstein'
'''

In [30]:
Albert_Einstein_q = list(cursor.execute(aa_quotes))
Albert_Einstein_q

[('“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'Albert Einstein'),
 ('“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'Albert Einstein'),
 ('“Try not to become a man of success. Rather become a man of value.”',
  'Albert Einstein'),
 ("“If you can't explain it to a six year old, you don't understand it yourself.”",
  'Albert Einstein'),
 ('“If you want your children to be intelligent, read them fairy tales. If you want them to be more intelligent, read them more fairy tales.”',
  'Albert Einstein'),
 ('“Logic will get you from A to Z; imagination will get you everywhere.”',
  'Albert Einstein'),
 ('“Any fool can know. The point is to understand.”', 'Albert Einstein'),
 ('“Life is like riding a bicycle. To keep your balance, you must keep moving.”',
  'Albert Einstein'),
 ('“If I were not a physicist, I would probably be a mu

In [31]:
tag_quotes_q = '''
    SELECT q.text, t.tag 
    FROM quotes q
        LEFT JOIN quotes_tags qt
            ON q.id = qt.quote_id
        LEFT JOIN tags t
            ON t.id = qt.tag_id
        WHERE t.tag IN ('humor', 'books') LIMIT 5
'''

In [32]:
tags_output = list(cursor.execute(tag_quotes_q))
tags_output

[('“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'humor'),
 ('“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'books'),
 ('“A day without sunshine is like, you know, night.”', 'humor'),
 ('“Good friends, good books, and a sleepy conscience: this is the ideal life.”',
  'books'),
 ('“Anyone who thinks sitting in church can make you a Christian must also think that sitting in a garage can make you a car.”',
  'humor')]

### Удаление таблиц из базы

> `удаление` таблиц

In [33]:
# drop_tables_query = '''
#     DROP TABLE quotes;
#     DROP TABLE authors;
#     DROP TABLE tags;
#     DROP TABLE quotes_authors;
#     DROP TABLE quotes_tags;
# '''

In [34]:
# execute_query(drop_tables_query, cursor, conn, 'script')

> `список` таблиц

In [37]:
show_tables_query = '''
    SELECT * FROM sqlite_master
    WHERE type = 'table'
'''

In [39]:
tables = list(cursor.execute(show_tables_query))
# tables